# Ejercicio 8 

Toma tu número $n$ de la lista publicada para este ejercicio. 

1. Pasa algunos tests de primalidad para ver si n es compuesto.
2. En caso que tu $n$ sea probable primo, factoriza $n+1$ encontrando certificados de primalidad para los factores mayores de $10000$.
3. Con $P = 1$, encuentra el menos $Q \ge 2$ tal que definan una sucesión de Lucas que certifique la primalidad de $n$ 

## Apartado 1

In [1]:
using Primes

In [2]:
n = 2783584280956932824984484769172860189421

2783584280956932824984484769172860189421

In [3]:
function simbolo_jacobi(a, n)
	if mod(n, 2) == 0
		throw("n debe ser impar")
	end
	
	t = 1 
	m = abs(n)
	b = mod(a, m)
	
	while a != 0
		while mod(a, 2) == 0
			a = a/2
			
			if mod(m, 8) in [3, 5]
				t = -t
			end
		end

		a, m = m, a

		if mod(a, 4) == 3 && mod(m, 4) == 3
			t = -t
		end

		a = mod(a, m)
	end

	if m == 1
		return t
	else
		return 0
	end
end

function test_solovay_strassen(a, n, salida = false)
	if salida
		println("Test de Solovay Strassen para a = ", a, ", n = ", n)
	end

	powmod = powermod(a, div(n-1, 2), n)
	
	if powmod ∉ [1, n-1]
		if salida
			println("\ta^{(n-1)/2} mod n = ", powmod, ", que no es congruente con 1 o -1. Falla.")
		end
		return false
	end

	if salida
		println("\ta^{(n-1)/2} mod n = ", powmod, ". Jacobi = ", simbolo_jacobi(a, n), "\n")
	end

	return mod(simbolo_jacobi(a, n), n)	== powmod
end

function lucas_lehmer(n)
	factores = collect(keys(factor(n-1)))

	for a in 1:(n-1)

		if powermod(a, n-1, n) == 1
			resultado = map(q -> powermod(a, div(n-1, q), n), factores)
			if 1 ∉ resultado
				return true
			else
				posicion = first(findall(x -> x == 1, resultado))
			end
		end
	end

	return false
end

lucas_lehmer (generic function with 1 method)

In [4]:
primos = [2, 3, 5, 7, 9, 11, 13, 17]

8-element Vector{Int64}:
  2
  3
  5
  7
  9
 11
 13
 17

In [5]:
map(a -> test_solovay_strassen(a, n), primos)

8-element Vector{Bool}:
 1
 1
 1
 1
 1
 1
 1
 1

In [6]:
# Julia utiliza por defecto el algoritmo de Miller-Rabin para probar si un número es primo.
# Como esta práctica no va sobre los tests en sí mismos, utilizo esta función del lenguaje, 
# puesto que la mía tenía fallos.
isprime(n)

true

In [7]:
lucas_lehmer(n)

true

El resultado de estos tests nos dicen que nuestro $n$ es un posible primo.

## Apartado 2

## Apartado 3

## Temporal

In [29]:
function ρ_Polard(n, f, t = 1000, x0 = 2)
	# En caso en el que sea divisible por dos, es posible que falle, así que lo comprobamos a mano.
	if mod(n, 2) == 0
		return [[2, div(n, 2)], 0]
	end

	x = x0
	y = x
	i = 0

	while i < t
		i = i + 1
		x = mod(f(x), n)
		y = mod(f(f(y)), n)
		g = gcd(x - y, n)

		if 1 < g && g < n
			return [[g, div(n, g)], i]
		end
	end

	return []
end

function descomponer(n)
	por_descomponer = [n]
	irreducibles = []
	f = x -> x^2 + 1
	i = 0

	while length(por_descomponer) > 0
		println("\nPor descomponer: $por_descomponer")
		num = pop!(por_descomponer)

		# Aplica el test de Miller Rabin primero. Si sale posible primo, saca 
		# un certificado de primalidad con Lucas Lehmer.

		if isprime(num) && lucas_lehmer(num)
			println("\t-> $num es primo")
			push!(irreducibles, num)
		else
			println("\t-> Descomponiendo $num")
			resultado = ρ_Polard(num, f)
			i = i + resultado[2]
			por_descomponer = vcat(por_descomponer, resultado[1])
		end
	end

	return [sort(irreducibles), i]
end


descomponer (generic function with 1 method)

In [30]:
descomponer(n+1)


Por descomponer: BigInt[2783584280956932824984484769172860189422]
	-> Descomponiendo 2783584280956932824984484769172860189422

Por descomponer: BigInt[2, 1391792140478466412492242384586430094711]
	-> Descomponiendo 1391792140478466412492242384586430094711

Por descomponer: BigInt[2, 3, 463930713492822137497414128195476698237]
	-> Descomponiendo 463930713492822137497414128195476698237

Por descomponer: BigInt[2, 3, 263, 1763995108337726758545300867663409499]
	-> Descomponiendo 1763995108337726758545300867663409499

Por descomponer: BigInt[2, 3, 263, 2237, 788553915215792024383236865294327]
	-> Descomponiendo 788553915215792024383236865294327

Por descomponer: BigInt[2, 3, 263, 2237, 487, 1619207218102242349862909374321]
	-> Descomponiendo 1619207218102242349862909374321

Por descomponer: BigInt[2, 3, 263, 2237, 487, 487, 3324860817458403182470039783]
	-> Descomponiendo 3324860817458403182470039783

Por descomponer: BigInt[2, 3, 263, 2237, 487, 487, 275131, 12084646286526793354693]
	-> 

2-element Vector{Any}:
    Any[2, 3, 263, 487, 487, 2237, 275131, 12084646286526793354693]
 823

In [18]:
factor(n+1)

2 * 3 * 263 * 487^2 * 2237 * 275131 * 12084646286526793354693